In [83]:
import pandas as pd


water_quality_df = pd.read_csv('Overall-dataset-llda(west phyto) assumed_knn_imputed_final2.csv')
weather_df = pd.read_csv('output.csv')

water_quality_df['Phytoplankton (cells/ml)'] = water_quality_df['Phytoplankton (cells/ml)'].str.replace(',', '')

water_quality_df['Month'] = water_quality_df['Month'].astype(str)

# Correct typos in the 'Month' column
water_quality_df['Month'] = water_quality_df['Month'].replace({
    'Febuary': 'February',
    'Aug': 'August',
    'Sept': 'September',
    'Nov': 'November',
    'Dec': 'December'
}, regex=False)  # Use regex=False to avoid treating the keys as regular expressions

# Display the updated DataFrame to verify the changes

# List of columns to exclude from transformation
exclude_columns = ['Month', 'Wind', 'Condition']

# Function to remove non-numeric characters from each cell
def remove_non_numeric(value):
    if isinstance(value, str) and value not in exclude_columns:
        return re.sub(r'[^0-9.]', '', value)
    else:
        return value

# Apply the function to each cell in numeric columns of the dataframe
for column in weather_df.columns:
    if column not in exclude_columns:
        weather_df[column] = weather_df[column].apply(remove_non_numeric)

# Convert the numeric columns to numeric type
numeric_columns = [col for col in weather_df.columns if col not in exclude_columns]
weather_df[numeric_columns] = weather_df[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Define mappings for Wind column
wind_mapping = {
    'N': 1, 'NNE': 2, 'NE': 3, 'ENE': 4,
    'E': 5, 'ESE': 6, 'SE': 7, 'SSE': 8,
    'S': 9, 'SSW': 10, 'SW': 11, 'WSW': 12,
    'W': 13, 'WNW': 14, 'NW': 15, 'NNW': 16,
    'VAR': 17, 'CALM': 18
}

# Define mappings for Condition column
condition_mapping = {
    'Fair': 1, 'Mostly Cloudy': 2, 'Partly Cloudy': 3, 'Cloudy': 4,
    'Light Rain': 5, 'Light Rain Shower': 6, 'Rain': 7, 'Heavy Rain': 8,
    'Thunder': 9, 'Light Rain with Thunder': 10, 'T-Storm': 11,
    'Heavy Rain Shower': 12, 'Rain Shower': 13, 'Showers in the Vicinity': 14,
    'Thunder in the Vicinity': 15, 'Mostly Cloudy / Windy': 16,
    'Fair / Windy': 17, 'Partly Cloudy / Windy': 18, 'Rain / Windy': 19,
    'Light Rain Shower / Windy': 20, 'Heavy Rain / Windy': 21
}

# Apply mappings to Wind and Condition columns
weather_df['Wind'] = weather_df['Wind'].map(wind_mapping)
weather_df['Condition'] = weather_df['Condition'].map(condition_mapping)



# Define a function to compute the mode
def compute_mode(series):
    return series.mode().iloc[0] if not series.mode().empty else None

# Group by 'Year' and 'Month'
grouped = weather_df.groupby(['Year', 'Month'])

# Aggregate to get the mean for all columns except 'Wind' and 'Condition'
# and the mode for 'Wind' and 'Condition'
weather_monthly_stats = grouped.agg({
    'Wind': compute_mode,
    'Condition': compute_mode,
    'Time': 'mean',
    'Temperature': 'mean',
    'Dew Point' : 'mean',
    'Humidity': 'mean',
    'Wind Speed': 'mean',
    'Wind Gust': 'mean',
    'Pressure': 'mean',
    'Precip.': 'mean'
}).reset_index()

# Print the result
print(weather_monthly_stats)


# Merge datasets on 'Month' and 'Year'
merged_df = pd.merge(water_quality_df, weather_monthly_stats, on=['Month', 'Year'])

print(merged_df['Month'].unique())

# Step 8: Save the updated dataset
output_file_path = 'merged_dataset.csv'
merged_df.to_csv(output_file_path, index=False)


     Year      Month  Wind  Condition        Time  Temperature  Dew Point  \
0    2013      April   6.0          1  650.000000    30.375000  22.208333   
1    2013     August   5.0          2  650.000000    29.208333  24.208333   
2    2013   December  18.0          2  650.000000    26.625000  24.208333   
3    2013   February   6.0          1  652.173913    27.434783  20.130435   
4    2013    January   1.0          2  650.000000    25.875000  21.458333   
..    ...        ...   ...        ...         ...          ...        ...   
112  2022      March   6.0          1  650.000000    28.666667  20.333333   
113  2022        May   6.0          1  650.000000    30.875000  22.375000   
114  2022   November  17.0          3  650.000000    28.208333  24.250000   
115  2022    October  13.0          1  650.000000    29.583333  22.750000   
116  2022  September  13.0          2  650.000000    28.583333  25.375000   

      Humidity  Wind Speed  Wind Gust     Pressure  Precip.  
0    62.04166

In [29]:
import pandas as pd

# Read the CSV file with ISO-8859-1 encoding
df = pd.read_csv('Overall-weather-dataset(csv).csv', encoding='iso-8859-1')

# Write the CSV file with UTF-8 encoding
df.to_csv('output.csv', encoding='utf-8', index=False)
